In [ ]:
# Import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
import seaborn as sns

# Make pandas dataframes prettier
from IPython.display import display, HTML

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [ ]:
print('hi')

In [ ]:
auth.authenticate_user()

In [ ]:
project_id='ai-harms'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

In [ ]:
# create a connection to the database
client = bigquery.Client(project='physionet-data')

# load the dataset list
datasets = client.list_datasets()

# iterate the datasets list
for dataset in datasets:
    did = dataset.dataset_id
    # print the dataset name
    print('Dataset "{}" has the following tables: '.format(did))
    # iterate the tables on the dataset
    for table in client.list_tables(client.dataset(did)):
        # print the table name
        print('- {}'.format(table.table_id))

In [ ]:
import pandas_gbq

# Helper function to read data from BigQuery into a DataFrame.
def run_query(query, project_id, location=None):
    return pandas_gbq.read_gbq(query, project_id=project_id, location=location, use_bqstorage_api=True)

In [ ]:
query = """
SELECT *
FROM `physionet-data.eicu_crd_demo.patient`
"""
project_id = 'ai-harms'  # Replace with your actual project ID
location = 'US'  # Replace with the actual location of your dataset if necessary

patient_data = run_query(query, project_id, location)

In [ ]:
patient_data.columns.to_list()

In [ ]:
patient_data

# Patient Demographics


## Ethnicity

In [ ]:
patient_data['ethnicity'] = patient_data['ethnicity'].apply(lambda x: 'Other/Unknown' if x == '' else x)

In [ ]:
patient_data['ethnicity'].value_counts()

In [ ]:
# Getting the counts of each ethnicity
ethnicity_counts = patient_data['ethnicity'].value_counts()

# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(ethnicity_counts, labels=ethnicity_counts.index, autopct='%1.1f%%', startangle=140, colors=plt.cm.Set2(range(len(ethnicity_counts))))
plt.title('Distribution of Ethnicity')
plt.show()

## Age

In [ ]:
patient_data['age'] = pd.to_numeric(patient_data['age'], errors='coerce')

In [ ]:
patient_data['age'].describe()

In [ ]:
patient_data['age'].unique()

In [ ]:
patient_data['age'].isna().sum()

In [ ]:
plt.figure(figsize=(10, 6))
plot = sns.histplot(patient_data['age'], bins=30, color='skyblue')
plt.title('Distribution of Patient Ages')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

## Gender

In [ ]:
patient_data['gender'].value_counts()

In [ ]:
gender_counts = patient_data['gender'].value_counts()
plt.figure(figsize=(8, 8))
plt.pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("Set3"))
plt.title('Gender Distribution')
plt.show()

## Other Analysis

In [ ]:
pid_counts = patient_data['uniquepid'].value_counts()

In [ ]:
patient_data['uniquepid'].describe()

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(pid_counts, bins=30, color='skyblue')  # Adjust bins as needed
plt.title('Histogram of Patient ID Frequencies')
plt.xlabel('Frequency of each unique patient ID')
plt.ylabel('Number of Patient IDs')
plt.show()

In [ ]:
# some ppl have been admitted multiple times, should these patients be put into a separate cohort (/ do they possible skew ml model results?)

# Apache Patient Results


In [ ]:
query = """
SELECT *
FROM `physionet-data.eicu_crd_demo.apachepatientresult`
"""
project_id = 'ai-harms'  # Replace with your actual project ID
location = 'US'  # Replace with the actual location of your dataset if necessary

apache_patient_results_df = run_query(query, project_id, location)
print(apache_patient_results_df.head())

In [ ]:
apache_patient_results_df

In [ ]:
apache_patient_results_df.columns.to_list()

In [ ]:
apache_patient_results_df['actualhospitalmortality'].value_counts()

In [ ]:
apache_patient_results_df['patientunitstayid'].value_counts()

### Does mortality disproportionately affect any demographic group?

In [ ]:
merged_df = apache_patient_results_df.merge(patient_data, on='patientunitstayid', how='inner')

In [ ]:
bins = [0, 18, 35, 65, 85, 100]  # Define age bins
labels = ['0-18', '19-35', '36-65', '66-85', '86-100']
merged_df['age_group'] = pd.cut(merged_df['age'], bins=bins, labels=labels, right=False)

# Calculate mortality rates
mortality_summary = merged_df.groupby(['age_group', 'gender', 'ethnicity'])['actualhospitalmortality'].value_counts(normalize=True).unstack().fillna(0)

In [ ]:
mortality_summary['EXPIRED'].plot(kind='bar', stacked=True, figsize=(30, 7))
plt.title('Mortality Rates Across Demographic Groups')
plt.xlabel('Demographic Groups')
plt.ylabel('Mortality Rate')
plt.xticks(rotation=90)
plt.show()

# Diagnosis

In [ ]:
query = """
SELECT *
FROM `physionet-data.eicu_crd_demo.diagnosis`
"""
project_id = 'ai-harms'  # Replace with your actual project ID
location = 'US'  # Replace with the actual location of your dataset if necessary

diagnosis = run_query(query, project_id, location)
print(diagnosis.head())

In [ ]:
icd9_counts = diagnosis['icd9code'].value_counts()

In [ ]:
# curious to see which groups of people typically have the least common icd diagnosis / no diagonsis at all?

In [ ]:
# Set aesthetic style
sns.set(style="whitegrid")

# Plotting the top 20 ICD-9 codes
top_icd9_codes = icd9_counts.head(20)  # Adjust the number as needed

plt.figure(figsize=(10, 8))  # Adjust the figure size as needed
top_icd9_codes.plot(kind='barh', color='skyblue')  # Horizontal bar plot
plt.title('Top 20 ICD-9 Codes Frequency')
plt.xlabel('Frequency')
plt.ylabel('ICD-9 Code')
plt.gca().invert_yaxis()  # Invert the y-axis to have the highest values on top
plt.show()

In [ ]:
icd9_counts

#### ICD 9 codes

038.9, A41.9 - Septicemia

518.81, J96.00 - Acute respiratory failure

427.31, I48.0 - Atrial fibrillation

584.9, N17.9 - Acute renal failure

401.9, I10 - Essential (primary) hypertension

486, J18.9 - Pneumonia

491.20, J44.9 - Chronic obstructive pulmonary disease

780.09, R41.82 - Altered mental status

428.0, I50.9 - Congestive heart failure

458.9, I95.9 - Hypotension

518.82 - Other specified respiratory conditions

288.8, D72.829 - Elevated white blood cell count

578.9, K92.2 - Gastrointestinal hemorrhage

511.9, J91.8 - Pleural effusion in conditions classified elsewhere

995.90 - Unspecified adverse effect of unspecified drug, medicament and
biological substance

799.02, J96.91 - Respiratory arrest

276.2, E87.2 - Acidosis

427.5, I46.9 - Cardiac arrest



244.9, E03.9 - Hypothyroidism

# Lab

In [ ]:
query = """
SELECT *
FROM `physionet-data.eicu_crd_demo.lab`
"""
project_id = 'ai-harms'  # Replace with your actual project ID
location = 'US'  # Replace with the actual location of your dataset if necessary

lab = run_query(query, project_id, location)
print(lab.head())

In [ ]:
lab.columns.to_list()

In [ ]:
lab_name_counts = lab['labname'].value_counts()

In [ ]:
# Set aesthetic style
sns.set(style="whitegrid")
plt.figure(figsize=(10, 30))  # Adjust the figure size as needed
lab_name_counts.plot(kind='barh', color='skyblue')  # Horizontal bar plot
plt.title('Lab Frequency')
plt.xlabel('Frequency')
plt.ylabel('Lab Name')
plt.gca().invert_yaxis()  # Invert the y-axis to have the highest values on top
plt.show()

In [ ]:
# analyze test results for top 10 labs

In [ ]:
top_10_labs = lab_name_counts.head(10).index.tolist()
top_10_lab_results = lab[lab['labname'].isin(top_10_labs)]
sns.set(style="whitegrid")

# Create a box plot for each lab test result
plt.figure(figsize=(12, 10))  # You can adjust the figure size as needed
sns.boxplot(y='labname', x='labresult', data=top_10_lab_results, orient='h', palette="Set2")
plt.title('Summary of Top 10 Lab Test Results')
plt.ylabel('Lab Test')
plt.xlabel('Test Result')
plt.show()